In [50]:
#修正异常
def abnormal_value_correct(df):
    #形式为带空值的日销量df
    df=df.copy()
    df['mean']=df.mean(axis=1)
    error_dict={}
    c=0
    df['mean'].fillna(0,inplace=True)
    for i in df.index:
        for j in df.columns:
            if df.loc[i,j]>(df.loc[i,'mean']*10):
                error_dict[j]=i
                c+=1
                df.loc[i,j]=df.loc[i,'mean']*20
    return df,error_dict ,c     

In [57]:
import pandas as pd
from datetime import date, timedelta

In [55]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [253]:
#周末销量占比
def weekday_ratio(df,t2017,week):
    windows=get_timespan(df, t2017, 7*week, 7*week,freq='D').sum(axis=1).values
    weekday=get_timespan(df, t2017, 7*week, 5*week,freq='B').sum(axis=1).values
    return weekday/windows

In [156]:
#某日期前的最早记录日期距今
def longest_counts(df, t2017):
    l=[]
    for i in df.index:
        j=0
        while float(get_timespan(df.loc[i,:], t2017, j+1, 1,freq='D').values)==0:
            j=j+1
        l.append(j)
    return l

In [172]:
def onpromotion_day(df_sales,df_onpromotion,dt, minus, periods, freq='D'):
    return get_timespan(df_sales,dt, minus, periods, freq=freq)[get_timespan(df_onpromotion,dt, minus, periods, freq=freq).fillna(False).values]

In [180]:
def not_onpromotion_day(df_sales,df_onpromotion,dt, minus, periods, freq='D'):
    df_not_onpromotion=df_onpromotion.replace([True,False],[1,0]).replace([1,0],[False,True])
    return get_timespan(df_sales,dt, minus, periods, freq=freq)[get_timespan(df_not_onpromotion,dt, minus, periods, freq=freq).fillna(False).values]

In [195]:
train_sales_fillna=train_sales.fillna(0)

In [254]:
def prepare_dataset(t2017, is_train=True):
    
    #某日期前14,60,140天的累积打折次数
    X = pd.DataFrame({
        #某日期前n天内的除0外的平均单价
        "mean_2_2017": get_timespan(train_sales, t2017, 2, 2).mean(axis=1).values,
        "mean_4_2017": get_timespan(train_sales, t2017, 4, 4).mean(axis=1).values,
        "mean_7_2017": get_timespan(train_sales, t2017, 7, 7).mean(axis=1).values,
        "mean_10_2017": get_timespan(train_sales, t2017, 10, 10).mean(axis=1).values,
        "mean_14_2017": get_timespan(train_sales, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(train_sales, t2017, 30,30).mean(axis=1).values,
        "mean_60_2017": get_timespan(train_sales, t2017,60,60).mean(axis=1).values,
        #某日期前n天累积销量
        "cumsum_1_2017": get_timespan(train_sales, t2017, 1, 1).values.ravel(),
        "cumsum_2_2017": get_timespan(train_sales, t2017, 2, 2).sum(axis=1).values,
        "cumsum_4_2017": get_timespan(train_sales, t2017, 4, 4).sum(axis=1).values,
        "cumsum_7_2017": get_timespan(train_sales, t2017, 7, 7).sum(axis=1).values,
        "cumsum_10_2017": get_timespan(train_sales, t2017, 10, 10).sum(axis=1).values,
        "cumsum_14_2017": get_timespan(train_sales, t2017, 14, 14).sum(axis=1).values,
        "cumsum_30_2017": get_timespan(train_sales, t2017, 30,30).sum(axis=1).values,
        "cumsum_60_2017": get_timespan(train_sales, t2017, 60,60).sum(axis=1).values,     
        #某日期前 天内的记录条目数
        "record_1": get_timespan(train_sales, t2017, 1, 1).count(axis=1).values,
        "record_2": get_timespan(train_sales, t2017, 2, 2).count(axis=1).values,
        "record_4": get_timespan(train_sales, t2017, 4, 4).count(axis=1).values,
        "record_7": get_timespan(train_sales, t2017, 7, 7).count(axis=1).values,
        "record_10": get_timespan(train_sales, t2017, 10, 10).count(axis=1).values,
        "record_14": get_timespan(train_sales, t2017, 14, 14).count(axis=1).values,
        "record_30": get_timespan(train_sales, t2017, 30,30).count(axis=1).values,
        "record_60": get_timespan(train_sales, t2017, 60,60).count(axis=1).values,
        #周内销量占比1,2,4,6,10周
        "weekday_ratio_1":weekday_ratio(train_sales, t2017,1),
        "weekday_ratio_2":weekday_ratio(train_sales, t2017,2),
        "weekday_ratio_4":weekday_ratio(train_sales, t2017,4),
        "weekday_ratio_6":weekday_ratio(train_sales, t2017,6),
        "weekday_ratio_10":weekday_ratio(train_sales, t2017,10),
        #某日期前n天的标准差
        "std_2": get_timespan(train_sales, t2017, 2, 2).std(axis=1).values,
        "std_4": get_timespan(train_sales, t2017, 4, 4).std(axis=1).values,
        "std_7": get_timespan(train_sales, t2017, 7, 7).std(axis=1).values,
        "std_10": get_timespan(train_sales, t2017, 10, 10).std(axis=1).values,
        "std_14": get_timespan(train_sales, t2017, 14, 14).std(axis=1).values,
        "std_30": get_timespan(train_sales, t2017, 30,30).std(axis=1).values,
        "std_60": get_timespan(train_sales, t2017,60,60).std(axis=1).values, 
        #某日期前的最早记录日期距今
        "longest_records":longest_counts(train_sales, t2017),
        #窗口打折率
        "promo_7":(get_timespan(train_onpromotion,t2017, 7, 7).sum(axis=1)/7).values,
        "promo_14": (get_timespan(train_onpromotion,t2017, 14, 14).sum(axis=1)/14).values,
        "promo_30": (get_timespan(train_onpromotion, t2017,30,30).sum(axis=1)/30).values,
        "promo_60": (get_timespan(train_onpromotion, t2017,60,60).sum(axis=1)/60).values,
        #打折期历史销量均值与未打折的平均销量比
        "onpromotion_sales_ratio_7":onpromotion_day(train_sales,train_onpromotion, t2017, 7, 7).mean(axis=1).values/not_onpromotion_day(train_sales,train_onpromotion, t2017, 7, 7).mean(axis=1).values,
        "onpromotion_sales_ratio_14":onpromotion_day(train_sales,train_onpromotion, t2017, 14, 14).mean(axis=1).values/not_onpromotion_day(train_sales,train_onpromotion, t2017, 14, 14).mean(axis=1).values,
        "onpromotion_sales_ratio_30":onpromotion_day(train_sales,train_onpromotion, t2017,30,30).mean(axis=1).values/not_onpromotion_day(train_sales,train_onpromotion, t2017, 30,30).mean(axis=1).values,
        "onpromotion_sales_ratio_60":onpromotion_day(train_sales,train_onpromotion, t2017,60,60).mean(axis=1).values/not_onpromotion_day(train_sales,train_onpromotion, t2017,60,60).mean(axis=1).values,
        #窗口最大值与均值的比
        "mean_2_2017": get_timespan(train_sales, t2017, 7, 7).max(axis=1).values/get_timespan(train_sales, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(train_sales, t2017, 14, 14).max(axis=1).values/get_timespan(train_sales, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(train_sales, t2017, 30,30).max(axis=1)/get_timespan(train_sales, t2017, 30,30).mean(axis=1).values,
        "mean_60_2017": get_timespan(train_sales, t2017, 60,60).max(axis=1)/get_timespan(train_sales, t2017, 60,60).mean(axis=1).values
    })
    #某日期前一周的历史周X的4.10周平均单价占整周的比例
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(train_sales_fillna, t2017, 28-i, 4, freq='7D').mean(axis=1).values/get_timespan(train_sales_fillna, t2017, 28, 4, freq='W').mean(axis=1).values
        X['mean_10_dow{}_2017'.format(i)] = get_timespan(train_sales_fillna, t2017, 70-i, 10, freq='7D').mean(axis=1).values/get_timespan(train_sales_fillna, t2017, 70, 10, freq='W').mean(axis=1).values

    # 某日期后1-15天的打折信息
    for i in range(16): 
        
        X["promo_{}".format(i)] = train_onpromotion[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    #控制产生新df的开关
    if is_train:
        #特定日期后半个月的每日单价
        y = train_sales_fillna[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X


In [255]:
X=prepare_dataset(date(2017,8,16),is_train=False)

ValueError: operands could not be broadcast together with shapes (74456,) (721001,) 

In [256]:
train_sales.shape

(167515, 227)

In [257]:
onpromotion_day(train_sales,train_onpromotion, t2017, 7, 7).mean(axis=1).values/not_onpromotion_day(train_sales,train_onpromotion, t2017, 7, 7).mean(axis=1).values

ValueError: operands could not be broadcast together with shapes (74489,) (719344,) 

In [269]:
onpromotion_day(train_sales_fillna,train_onpromotion_fillna, t2017, 7, 7).reindex(train_sales.index).fillna(0)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [270]:
onpromotion_day(train_sales_fillna,train_onpromotion_fillna, t2017, 7, 7)

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         105577      0.000000    0.693147    0.693147    0.000000   
          108696      0.693147    1.386294    1.098612    0.693147   
          108701      0.000000    1.386294    0.693147    0.693147   
          108701      0.000000    1.386294    0.693147    0.693147   
          108701      0.000000    1.386294    0.693147    0.693147   
          108701      0.000000    1.386294    0.693147    0.693147   
          108701      0.000000    1.386294    0.693147    0.693147   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          108797      1.098612    1.609438    2.079442    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          114790      1.609438    1.386294    1.386294    1.098612   
          115891      2.484907    0.000000    1.098612    0.693147   
          115891      2.484907    0.000000    1.098612    0.693147   
          115891      2.484907    0.000000    1.098612    0.693147   
          115893      2.484907    2.079442    1.609438    0.693147   
          115893      2.484907    2.079442    1.609438    0.693147   
          122419      0.693147    2.302585    0.693147    0.000000   
          122419      0.693147    2.302585    0.693147    0.000000   
          127547      1.386294    0.000000    1.098612    1.609438   
          158842      0.000000    0.000000    0.000000    0.000000   
          159156      1.531260    1.945481    3.397055    2.190871   
...                        ...         ...         ...         ...   
54        2061025     3.465736    2.564949    3.688879    3.258097   
          2061025     3.465736    2.564949    3.688879    3.258097   
          2061025     3.465736    2.564949    3.688879    3.258097   
          2061025     3.465736    2.564949    3.688879    3.258097   
          2061025     3.465736    2.564949    3.688879    3.258097   
          2061025     3.465736    2.564949    3.688879    3.258097   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061033     3.433987    2.995732    3.091042    3.218876   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2061214     6.025866    5.075174    5.433722    5.141664   
          2083837     0.000000    2.564949    2.397895    0.000000   
          2083837     0.000000    2.564949    2.397895    0.000000   
          2083837     0.000000    2.564949    2.397895    0.000000   
          2083859     0.000000    3.218876    0.000000    0.000000   
          2086882     1.098612    0.693147    1.386294    1.386294   
          

In [ ]:
for i in [7,14,30,60]:
    for j in get_timespan(train_onpromotion_fillna,t2017, i, i).index:
        row=get_timespan(train_sales,t2017, i, i).loc[j,:]
        on=row[row[get_timespan(train_onpromotion_fillna,t2017, i, i).loc[j,:].values]]
        off=row[row[get_timespan(train_onpromotion_fillna,t2017, i ,i).iloc[j,:].replace([True,False],[1,0]).replace([1,0],[False,True]).values]]

In [312]:
windows=get_timespan(train_sales_fillna,t2017, 7, 7).iloc[1,:]
windows[windows[get_timespan(train_onpromotion_fillna,t2017, 7 ,7).iloc[1,:].values]].values/3


array([], dtype=float64)

In [305]:
windows

date
2017-08-08         NaN
2017-08-09    1.098612
2017-08-10         NaN
2017-08-11         NaN
2017-08-12         NaN
2017-08-13         NaN
2017-08-14         NaN
Name: (1, 99197), dtype: float64

In [304]:
get_timespan(train_onpromotion_fillna,t2017, 7 ,7).iloc[3,:].replace([True,False],[1,0]).replace([1,0],[False,True]).values

array([ True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [292]:
train_onpromotion_fillna

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
          105575         False       False       False       False   
          105577         False       False       False       False   
          105693         False       False       False       False   
          105737         False       False       False       False   
          105857         False       False       False       False   
          106716         False       False       False       False   
          108079         False       False       False       False   
          108634         False       False       False       False   
          108696         False       False       False       False   
          108698         False       False       False       False   
          108701         False       False       False       False   
          108786         False       False       False       False   
          108797         False       False       False       False   
          108831         False       False       False       False   
          108862         False       False       False       False   
          108952         False       False       False       False   
          111223         False       False       False       False   
          111397         False       False       False       False   
          112830         False       False       False       False   
          114778         False       False       False       False   
          114790         False       False       False       False   
          114799         False       False       False       False   
          114800         False       False       False       False   
          115267         False       False       False       False   
          115611         False       False       False       False   
...                        ...         ...         ...         ...   
54        2081056        False       False       False       False   
          2081064        False       False       False       False   
          2081095        False       False       False       False   
          2081142        False       False       False       False   
          2081161        False       False       False       False   
          2083837        False       False       False       False   
          2083859        False       False       False       False   
          2084278        False       False       False       False   
          2084557        False       False       False       False   
          2086882        False       False       False       False   
          2087374        False       False       False       False   
          2087409        False       False       False       False   
          2087543        False       False       False       False   
          2087567        False       False       False       False   
          2087933        False       False       False       False   
          2087978        False       False       False       False   
          2088922        False       False       False       False   
          2089036        False       False       False       False   
          2089339        False       False       False       False   
          2099230        False       False       False       False   
          2101795        False       False       False       False   
          2103250        False       False       False       False   
          2105347        False       False       False       False   
          2106464        False       False       False       False   
          

In [293]:
train_onpromotion

date               2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05  \
store_nbr item_nbr                                                          
1         96995          None       None       None       None       None   
          99197          None       None      False      False      False   
          103520         None      False      False       None      False   
          103665         None       None       None      False      False   
          105574         None       None       True      False      False   
          105575         None      False      False      False      False   
          105577         None      False      False      False      False   
          105693         None       None      False      False       None   
          105737         None      False      False      False      False   
          105857         None       None      False      False      False   
          106716         None       None      False      False      False   
          108079         None      False      False       None      False   
          108634         None       None      False      False       None   
          108696         None       None       None       None       None   
          108698         None       None      False      False       None   
          108701         None       None      False      False      False   
          108786         None      False      False       None      False   
          108797         None      False      False      False      False   
          108831         None       None       None       None       None   
          108862         None      False      False      False      False   
          108952         None      False      False      False      False   
          111223         None      False      False      False      False   
          111397         None       None       None      False      False   
          112830         None       None      False      False      False   
          114778         None       None       None       None       None   
          114790         None      False      False      False      False   
          114799         None      False       None      False      False   
          114800         None      False      False      False      False   
          115267         None      False      False      False      False   
          115611         None       None      False      False      False   
...                       ...        ...        ...        ...        ...   
54        2081056        None       None       None       None       None   
          2081064        None       None       None       None       None   
          2081095        None       None       None       None       None   
          2081142        None       None       None       None       None   
          2081161        None       None       None       None       None   
          2083837        None       None       None       None       None   
          2083859        None       None       None       None       None   
          2084278        None       None       None       None       None   
          2084557        None       None       None       None       None   
          2086882        None       None       None       None       None   
          2087374        None       None       None       None       None   
          2087409        None       None       None       None       None   
          2087543        None       None       None       None       None   
          2087567        None       None       None       None       None   
          2087933        None       None       None       None       None   
          2087978        None       None       None       None       None   
          2088922        None       None       None       None       None   
          2089036        None       None       None       None       None   
          2089339        None       None       None       None       None